In [ ]:
# !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

In [ ]:
# !tar xzvf food-101.tar.gz

In [3]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3, InceptionV3, ResNet101
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print(tf.__version__)

2.4.1


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from google.colab import files

# files.upload()

In [ ]:
# train_path = '../input/food101/training'
# val_path = '../input/food101/validation'

In [ ]:
!unzip /content/drive/MyDrive/data/dset.zip

In [6]:
train_path = '/content/dset'

# Deep Learning

# Convolution neural network (CNN)

# Data Generator

In [11]:
img_size = 224
img_shape = (img_size,img_size,3)
batch_size = 32
epochs = 8
dropout_rate = 0.5
num_of_predict = 4

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='categorical'
)
# val_generator = train_datagen.flow_from_directory(
#     val_path,
#     target_size=(img_size,img_size),
#     batch_size=batch_size,
#     class_mode='categorical'
# )

Found 423 images belonging to 4 classes.


# Base model

## EfficientNet B3  
Tan, Mingxing, and Quoc Le. "Efficientnet: Rethinking model scaling for convolutional neural networks." International Conference on Machine Learning. PMLR, 2019.

In [10]:
efficientnet = EfficientNetB3(
    include_top=False, weights='imagenet',
    input_shape=img_shape, classes = num_of_predict,
    classifier_activation='softmax'
)

inputs = Input(shape=img_shape)
efficientnet = efficientnet(inputs)
pooling = layers.GlobalAveragePooling2D()(efficientnet)
dropout = layers.Dropout(dropout_rate)(pooling)
outputs = Dense(num_of_predict, activation="softmax")(dropout)
eff_model = Model(inputs=inputs, outputs=outputs)

43941888/43941136 [==============================] - 0s 0us/step


## InceptionV3  
Szegedy, Christian, et al. "Rethinking the inception architecture for computer vision." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.

In [12]:
inception = InceptionV3(
    include_top=False, weights='imagenet',
    input_shape=img_shape, classes = num_of_predict,
    classifier_activation='softmax')

inputs = Input(shape=img_shape)
inception = inception(inputs)
pooling = layers.GlobalAveragePooling2D()(inception)
dropout = layers.Dropout(dropout_rate)(pooling)
outputs = Dense(num_of_predict, activation="softmax")(dropout)
inception_model = Model(inputs=inputs, outputs=outputs)

87916544/87910968 [==============================] - 0s 0us/step


## ResNet101  
He, Kaiming, et al. "Deep residual learning for image recognition." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.

In [13]:
ResNet101 = ResNet101(
    include_top=False, weights='imagenet',
    input_shape=img_shape, classes = num_of_predict,
    classifier_activation='softmax'
)

inputs = Input(shape=img_shape)
ResNet101 = ResNet101(inputs)
pooling = layers.GlobalAveragePooling2D()(ResNet101)
dropout = layers.Dropout(dropout_rate)(pooling)
outputs = Dense(num_of_predict, activation="softmax")(dropout)
res_model = Model(inputs=inputs, outputs=outputs)

171450368/171446536 [==============================] - 1s 0us/step


In [14]:
res_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet101 (Functional)       (None, 7, 7, 2048)        42658176  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 8196      
Total params: 42,666,372
Trainable params: 42,561,028
Non-trainable params: 105,344
_________________________________________________________________


# Train

In [29]:
decay_steps = int(round(423/batch_size))*epochs
cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3)

callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

eff_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(cosine_decay), metrics=["accuracy"])

In [30]:
history = eff_model.fit(train_generator,
                  epochs = epochs)

Epoch 1/10
 8/14 [================>.............] - ETA: 5s - loss: 1.3824 - accuracy: 0.3564

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


14/14 [==============================] - 25s 801ms/step - loss: 1.2763 - accuracy: 0.4299
Epoch 2/10
14/14 [==============================] - 10s 707ms/step - loss: 0.6881 - accuracy: 0.8096
Epoch 3/10
14/14 [==============================] - 10s 720ms/step - loss: 0.3599 - accuracy: 0.9139
Epoch 4/10
14/14 [==============================] - 11s 701ms/step - loss: 0.2798 - accuracy: 0.9346
Epoch 5/10
14/14 [==============================] - 10s 701ms/step - loss: 0.2127 - accuracy: 0.9541
Epoch 6/10
14/14 [==============================] - 10s 716ms/step - loss: 0.1464 - accuracy: 0.9720
Epoch 7/10
14/14 [==============================] - 10s 694ms/step - loss: 0.1314 - accuracy: 0.9870
Epoch 8/10
14/14 [==============================] - 10s 699ms/step - loss: 0.1306 - accuracy: 0.9663
Epoch 9/10
14/14 [==============================] - 10s 728ms/step - loss: 0.1031 - accuracy: 0.9860
Epoch 10/10
14/14 [==============================] - 10s 710ms/step - loss: 0.0925 - accuracy: 0.9913


# Visualization

In [ ]:
accuracy = history.history['accuracy']

In [ ]:
accuracy
x = range(len(accuracy))
y2 = accuracy
plt.plot(x, y2)
plt.show()

In [ ]:
predictions = model.predict(augmented_images)
final_prediction = np.argmax(global_predictions)

def run_predictions_over_image_list(image_list, folder):
    predictions = []
    with tqdm(total=len(image_list)) as pbar:
        for image_filename in image_list:
            pbar.update(1)
            predictions.append(predict_and_vote(image_filename, folder))
    return predictions

validation_df["results"] = run_predictions_over_image_list(validation_df["image_id"], training_folder)

In [ ]:
true_positives = 0
prediction_distribution_per_class = {"0":{"0": 0, "1": 0, "2":0, "3":0, "4":0}, # class X class 사이즈 로
                                     "1":{"0": 0, "1": 0, "2":0, "3":0, "4":0},
                                     "2":{"0": 0, "1": 0, "2":0, "3":0, "4":0},
                                     "3":{"0": 0, "1": 0, "2":0, "3":0, "4":0},
                                     "4":{"0": 0, "1": 0, "2":0, "3":0, "4":0}}
number_of_images = len(validation_df)
for idx, pred in validation_df.iterrows():
    if int(pred["label"]) == pred.results:
        true_positives+=1
    prediction_distribution_per_class[str(pred["label"])][str(pred.results)] += 1
print("accuracy: {}%".format(true_positives/number_of_images*100))

In [ ]:
prediction_distribution_per_class

In [ ]:
heatmap_df = pd.DataFrame(columns={"groundtruth","prediction","value"})
for key in prediction_distribution_per_class.keys():
    for pred_key in prediction_distribution_per_class[key].keys():
        value = prediction_distribution_per_class[key][pred_key]/validation_df.query("label==@key").count()[0]
        heatmap_df = heatmap_df.append({"groundtruth":key,"prediction":pred_key,"value":value}, ignore_index=True)   

heatmap = heatmap_df.pivot(index='groundtruth', columns='prediction', values='value')
sns.heatmap(heatmap,cmap="Blues")